# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
D. Christie  ->  D. Christie  |  ['D. Christie']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']


Arxiv has 69 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.20158


extracting tarball to tmp_2504.20158...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.20290


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2504.20158/combined_final.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'combined_final.bbl' from 'tmp_2504.20158/combined_final.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
D. Christie  ->  D. Christie  |  ['D. Christie']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  SOSS_transmission_spectrum_exotedrf_visit1_visit2_differences.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=.48\textwidth]{ SOSS_transmission_spectrum_exotedrf_visit1_visit2_differences.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  G395H_transmission_spectrum_visit1_and_visit2_with_differences_exotedrf.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic

/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  spectrum_fit_visit1_visit2_20241210.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.9\textwidth]{ spectrum_fit_visit1_visit2_20241210.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  GJ_3090_b_atmosphere_SCARLET_ScCHIMERA.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.9\textwidth]{ GJ_3090_b_atmosphere_SCARLET_ScCHIMERA.pdf}
 

/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  GridRetrieval_v4_1sigCO.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.8\linewidth]{ GridRetrieval_v4_1sigCO.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 227 bibliographic references in tmp_2504.20428/references.bbl.
Error retrieving bib data for fulton_california-kepler_2017: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.20428-b31b1b.svg)](https://arxiv.org/abs/2504.20428) | **Escaping Helium and a Highly Muted Spectrum Suggest a Metal-Enriched Atmosphere on Sub-Neptune GJ3090b from JWST Transit Spectroscopy**  |
|| <mark>E.-M. Ahrer</mark>, et al. -- incl., <mark>D. Christie</mark>, <mark>C. Gapp</mark> |
|*Appeared on*| *2025-04-30*|
|*Comments*| *36 pages, 21 figures, accepted for publication in ApJL*|
|**Abstract**|            Sub-Neptunes, the most common planet type, remain poorly understood. Their atmospheres are expected to be diverse, but their compositions are challenging to determine, even with JWST. Here, we present the first JWST spectroscopic study of the warm sub-Neptune GJ3090b (2.13R$_\oplus$, Teq~700 K) which orbits an M2V star, making it a favourable target for atmosphere characterization. We observed four transits of GJ3090b; two each using JWST NIRISS/SOSS and NIRSpec/G395H, yielding wavelength coverage from 0.6-5.2 $\mu$m. We detect the signature of the 10833 Å metastable Helium triplet at a statistical significance of 5.5$\sigma$ with an amplitude of 434$\pm$79 ppm, marking the first such detection in a sub-Neptune with JWST. This amplitude is significantly smaller than predicted by solar-metallicity forward models, suggesting a metal-enriched atmosphere which decreases the mass-loss rate and attenuates the Helium feature amplitude. Moreover, we find that stellar contamination, in the form of the transit light source effect, dominates the NIRISS transmission spectra, with unocculted spot and faculae properties varying across the two visits separated in time by approximately six months. Free retrieval analyses on the NIRSpec/G395H spectrum find tentative evidence for highly muted features and a lack of CH4. These findings are best explained by a high metallicity atmosphere (>100x solar at 3$\sigma$ confidence, for clouds at $\sim \mu$bar pressures) using chemically-consistent retrievals and self-consistent model grids. Further observations of GJ3090b are needed for tighter constraints on the atmospheric abundances, and to gain a deeper understanding of the processes that led to its potential metal enrichment.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.20158-b31b1b.svg)](https://arxiv.org/abs/2504.20158) | **Microlensing events indicate that super-Earth exoplanets are common in Jupiter-like orbits**  |
|| W. Zang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-30*|
|*Comments*| *Published in Science*|
|**Abstract**|            Exoplanets classified as super-Earths are commonly observed on short period orbits, close to their host stars, but their abundance on wider orbits is poorly constrained. Gravitational microlensing is sensitive to exoplanets on wide orbits. We observed the microlensing event OGLE-2016-BLG-0007, which indicates an exoplanet with a planet-to-star mass ratio roughly double the Earth-Sun mass-ratio, on an orbit longer than Saturn's. We combine this event with a larger sample from a microlensing survey to determine the distribution of mass ratios for planets on wide orbits. We infer there are $\sim 0.35$ super-Earth planets per star on Jupiter-like orbits. The observations are most consistent with a bimodal distribution, with separate peaks for super-Earths and gas giants. We suggest that this reflects differences in their formation processes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.20290-b31b1b.svg)](https://arxiv.org/abs/2504.20290) | **FALCO: a Foundation model of Astronomical Light Curves for time dOmain astronomy**  |
|| X. Zuo, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-30*|
|*Comments*| **|
|**Abstract**|            Time-domain surveys have advanced astronomical research by revealing diverse variable phenomena, from stellar flares to transient events. The scale and complexity of survey data, along with the demand for rapid classification, present significant challenges for analysis. While machine learning offers solutions, most existing models are tailored to single tasks, struggle to generalize, and depend heavily on large, accurately labeled datasets. We introduce FALCO, a foundation model for astronomical light curve analysis in time-domain astronomy. This work presents the initial version of FALCO trained via self-supervised learning on unlabeled Kepler light curves using a Transformer-based architecture. The model has been evaluated on three distinct tasks and demonstrates strong generalization: achieving 95 percent accuracy in stellar variability classification across eight classes, an overall RMSE of 0.1305 dex in surface gravity estimation (notably improved to below 0.08 dex when log g is less than 1, and approximately 0.02 dex near log g equals 3), and 87 percent precision in flare identification. These results highlight the model's versatility and ability to learn generalizable representations from light curves, enabling straightforward adaptation to diverse tasks. We further analyze the impact of model scaling and sequence length, finding performance improves with larger models and longer input sequences. We also apply FALCO to derive surface gravity (log g) measurements for 179,732 Kepler stars from their light curves.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.20407-b31b1b.svg)](https://arxiv.org/abs/2504.20407) | **Discovery of a variable yellow supergiant progenitor for the Type IIb SN 2024abfo**  |
|| Z. Niu, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-30*|
|*Comments*| *5 Figures. Submitted to ApJL. Comments are welcome*|
|**Abstract**|            We report the discovery of a progenitor candidate for the Type IIb SN 2024abfo using multi-epoch pre-explosion images from the Hubble Space Telescope and the Dark Energy Camera Legacy Survey. The progenitor exhibited a ~0.7 mag decline in F814W from 2001 to 2013, followed by significant brightness and color fluctuations in the g, r and z bands. This is the first time that a variable progenitor has been found for a SN IIb. We suggest that the variability is caused by intrinsic changes in the progenitor star instead of varying obscuration by circumstellar dust. Our results show that the progenitor of SN 2024abfo was likely a yellow supergiant star with an initial mass of 12--14, 14--15 and 15--18 Msun for different assumed values of circumstellar reddening of E(B-V)_CSM = 0, 0.1 and 0.2 mag. Our study underscores the critical role of multi-epoch imaging surveys in revealing the final stages of core-collapse supernovae progenitors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.20428/./joint_posterior_SCARLET_POSEIDON_AURORA_resubmission.png', '', '', '', '', 'tmp_2504.20428/./PopulationPlot.png']
copying  tmp_2504.20428/./joint_posterior_SCARLET_POSEIDON_AURORA_resubmission.png to _build/html/
file not found 
file not found 
file not found 
file not found 
copying  tmp_2504.20428/./PopulationPlot.png to _build/html/
exported in  _build/html/2504.20428.md
    + _build/html/tmp_2504.20428/./joint_posterior_SCARLET_POSEIDON_AURORA_resubmission.png
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/tmp_2504.20428/./PopulationPlot.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mystar}{GJ 3090\xspace}$
$\newcommand{\myplanet}{GJ 3090 b\xspace}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\logX}[1]{\ensuremath{\log(\mathrm{X_{\ce{#1}}})}}$
$\newcommand{\logXratio}[2]{\ensuremath{\log(\mathrm{X_{\ce{#1}} / X_{\ce{#2}} })}}$
$\newcommand{\umontreal}{Department of Physics and Trottier Institute for Research on Exoplanets, Université de Montréal, Montréal, QC, Canada}$
$\newcommand{\mpia}{Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}$
$\newcommand{\thefigure}{A.\arabic{figure}}$</div>



<div id="title">

# ${Escaping Helium and a Highly Muted Spectrum Suggest a Metal-Enriched Atmosphere}$ on Sub-Neptune $\myplanet$ from JWST Transit Spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.20428-b31b1b.svg)](https://arxiv.org/abs/2504.20428)<mark>Appeared on: 2025-04-30</mark> -  _36 pages, 21 figures, accepted for publication in ApJL_

</div>
<div id="authors">

<mark>E.-M. Ahrer</mark>, et al. -- incl., <mark>L. Acuña</mark>, <mark>D. Christie</mark>, <mark>C. Gapp</mark>

</div>
<div id="abstract">

**Abstract:** Sub-Neptunes, the most common planet type, remain poorly understood. Their atmospheres are expected to be diverse, but their compositions are challenging to determine, even with JWST. Here, we present the first JWST spectroscopic study of the warm sub-Neptune $\myplanet$ (2.13 R $_\oplus$ , T $_\mathrm{eq, A=0.3}\sim$ 700 K) which orbits an M2V star, making it a favourable target for atmosphere characterization. We observed four transits of $\myplanet$ ; two each using JWST NIRISS/SOSS and NIRSpec/G395H, yielding wavelength coverage from 0.6 -- 5.2 $\microns$ . ${  We detect the signature of the 10833 Å metastable Helium triplet at a statistical significance of 5.5$\sigma$ with an amplitude of 434$\pm79$ ppm, marking the first such detection in a sub-Neptune with JWST. This amplitude is significantly smaller than predicted by solar-metallicity forward models, suggesting a metal-enriched atmosphere which decreases the mass-loss rate and attenuates the Helium feature amplitude.Moreover, we find that stellar contamination, in the form of the transit light source effect, dominates the NIRISS transmission spectra, with unocculted spot and faculae properties varying across the two visits separated in time by approximately six months.Free retrieval analyses on the NIRSpec/G395H spectrum find tentative evidence for highly muted features and a lack of $CH_4$.These findings are best explained by a high metallicity atmosphere ($>$100$\times$ solar at 3$\sigma$ confidence, for clouds at $\sim$\textmu bar pressures) using chemically-consistent retrievals and self-consistent model grids.Further observations of \myplanet are needed for tighter constraints on the atmospheric abundances, and to gain a deeper understanding of the processes that led to its potential metal enrichment.}$

</div>

<div id="div_fig1">

<img src="tmp_2504.20428/./joint_posterior_SCARLET_POSEIDON_AURORA_resubmission.png" alt="Fig19" width="100%"/>

**Figure 19. -** {  Joint and marginalized posterior distributions for the main atmospheric absorbers that NIRSpec/G395H is sensitive to, from the SCARLET (blue), POSEIDON (orange), and AURORA (green) retrievals performed on the visit 1+2 NIRSpec/G395H spectrum of GJ 3090 b. While POSEIDON marginalized over the potential contributions from unocculted spots, the SCARLET and AURORA results shown do not account for a TLS contribution, leading to overall higher water abundances. The contours correspond to 1 and 2$\sigma$ confidence (dark and light color shadings).} (*fig:corner_nirspec_molecules*)

</div>
<div id="div_fig2">

<img src="" alt="Fig12.1" width="25%"/><img src="" alt="Fig12.2" width="25%"/><img src="" alt="Fig12.3" width="25%"/><img src="" alt="Fig12.4" width="25%"/>

**Figure 12. -** Comparison of transmission spectra of $\myplanet$ from different reduction pipelines. _Top left:_\texttt{Eureka!} transmission spectra from both  NIRSpec/G395H visits. Visit 1 is displayed in the darker colour and Visit 2 in the lighter colour and the differences between them are shown in the bottom panel in black.
    _Bottom left:_ Combined (visit 1 \& visit 2) NIRSpec transmission spectra from \texttt{Eureka!}(orange) and \texttt{exoTEDRF}(turquoise). The latter reduction is vertically offset by their average difference of 40 ppm to allow for direct comparison.
    _Top right:_\texttt{NAMELESS} transmission spectra of $\myplanet$ from both NIRISS/SOSS visits. Visit 1 is displayed in the darker purple colour and Visit 2 in the lighter orange. The differences between the two visits are shown in the bottom panel in black, demonstrating an $\sim$30 ppm offset between the two visits.
    _Bottom right:_ Comparison of the \texttt{NAMELESS}(orange, purple) and \texttt{exoTEDRF}(turquoise, dark blue) transmission spectra for both NIRISS/SOSS visits. (*fig:other-reductions-differences*)

</div>
<div id="div_fig3">

<img src="tmp_2504.20428/./PopulationPlot.png" alt="Fig1" width="100%"/>

**Figure 1. -** GJ 3090 b in context with the population of sub-Neptune planets in equilibrium temperature vs. planet radius space. Markers are coloured by the effective temperature of the stellar host and marker sizes reflect the planet mass. The background shading represents the distribution of sub-Neptune planets from fulton_california-kepler_2017, highlighting the radius valley near 1.7 R$_\oplus$. Planets with published JWST near-infrared atmosphere spectra are outlined in black and labelled. Planets that will have JWST atmosphere observations by the end of Cycle 3 are outlined in grey. (*fig: Population*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.20428"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

472  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

23  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

20  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
